In [2]:
!pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.4 MB 653.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.4 MB 762.6 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.4 MB 722.1 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.4 MB 1.1 MB/s eta 0:00:02
   -------- ------------------------------- 0.3/1.4 MB 1.1 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.4 MB 1.6 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.4 MB 1.9 MB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 2.6 MB/s eta 0:00:01
   ------------------------------------ --- 1.2/1.4 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 3.0 MB/s eta 0:00:00
   ------------------------

In [ ]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 

In [ ]:
cust_data = pd.read_excel('Customer_Shopping_data.xlsx') 
cust_data.head() 


In [ ]:
# Remove the unnecessary spaces in the description
cust_data['Description'] = cust_data['Description'].str.strip() 

# Dropping the rows without any invoice number 
cust_data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
cust_data['InvoiceNo'] = cust_data['InvoiceNo'].astype('str') 


In [ ]:
cust_data

In [ ]:
cust_data['Country'].value_counts()

In [ ]:
# Transactions analysis and association in United Kingdom  

cust_data_UK = (cust_data[cust_data['Country'] =="United Kingdom"] 
		.groupby(['InvoiceNo', 'Description'])['Quantity'] 
		.sum().unstack().reset_index().fillna(0) 
		.set_index('InvoiceNo'))

In [ ]:
cust_data_UK

In [ ]:
def one_hot_encoding(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

In [ ]:
cust_data_UK_encoded = cust_data_UK.map(one_hot_encoding) 

In [ ]:
cust_data_UK_encoded

In [ ]:
# Building the model 
frq_items = apriori(cust_data_UK_encoded, min_support = 0.01, use_colnames = True) 

# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

In [ ]:
rules.head()

In [ ]:
# Experiment with different min_support values
for min_sup in [0.01, 0.02, 0.05]:
    frq_items = apriori(cust_data_UK_encoded, min_support=min_sup, use_colnames=True)
    rules = association_rules(frq_items, metric="lift", min_threshold=1)
    print(f"Number of rules with min_support {min_sup}: {len(rules)}")